In [1]:
import torch
import os
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing
import numpy as np
from datasets import load_metric
from transformers import RobertaTokenizerFast, RobertaConfig, RobertaForMaskedLM
from transformers import LineByLineTextDataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
from transformers import pipeline
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification, RobertaForTokenClassification
from transformers import PreTrainedTokenizerFast, AutoTokenizer
from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"

# If cpu -> Runtime -> Change runtime type
print(device)

cuda:0


In [2]:
if device != 'cpu':
  !nvidia-smi


Fri Nov 12 18:20:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 462.31       Driver Version: 462.31       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 1650   WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   39C    P8     2W /  N/A |    132MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [22]:
#train_path = '/content/no_bokmaal-ud-train.txt'
#train_path = '/content/oscar_deduplicated_no_part_3_100MB.txt'
batch_size=16
train_path='oscar_no_small.txt'

with open(train_path,'r', encoding='utf-8') as txt_file:
    read_file = txt_file.readlines()
    print(read_file[:3])


['Slett midlertidige internett filer og informasjonskapsler, og følg guiden til din browser på wikihow.com.Ytterligere aktører i primærhelsetjenesten og andre NHS-virksomheter ble infisert, inkludert legekontor.Læreren vår er så attraktiv, liksom detaljert morsom og alltid i godt gemytt. Altså, altså har Abiword to saker igjen bekk ta opp på møtet. Hutchins lyktes som kjent med bekk registrere ei domenenavn, der hindret adskillig av spredningen til utpressingsvaren. Du har tre dager på deg før kravet dobles. Det var liksom tjue alias noe sånt.\n', 'Minst 81 fra engelske helseforetak ble rammet. I membranen er det også porer som slipper ut og inn molekyler med annonse. Det er viktig bekk gi god synlighet i tåke. Ego er altså ikke sikker på at hun kommer. Det er ingen grunn til at avstanden er lengre, altså bilen beveger seg bedagelig gjennom tåken. Når det ble gjort oppmerksom for at du kjørte med tåkelys for når du kjører i trange rom, med mange svinger i mørket. Ego er ikke sikker for

In [4]:
!pip install transformers
!pip list | grep -E 'transformers|tokenizers'

'grep' is not recognized as an internal or external command,
operable program or batch file.


In [3]:

vocab_size = 20000

tokenizer = ByteLevelBPETokenizer(lowercase=True)
tokenizer.train(files=train_path, vocab_size=vocab_size, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

NameError: name 'train_path' is not defined

In [7]:
!mkdir norBERT
tokenizer.save_model("norBERT")

A subdirectory or file norBERT already exists.


['norBERT\\vocab.json', 'norBERT\\merges.txt']

In [8]:
#!rm -rf norBERT

In [9]:




tokenizer = ByteLevelBPETokenizer(
    "./norBERT/vocab.json",
    "./norBERT/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)


In [10]:

configuration = RobertaConfig(
    hidden_size = 512,
    num_attention_heads = 8,
    num_hidden_layers = 10,
    max_position_embeddings = 514,
    vocab_size = vocab_size
)
tokenizer = RobertaTokenizerFast.from_pretrained("./norBERT", max_len=512)
#model = RobertaForMaskedLM(configuration).to(device)
#configuration = model.config
model = RobertaForMaskedLM.from_pretrained("./NER_nor-finetuned-ner/checkpoint-500")
print(configuration)
print("Num parameters", model.num_parameters())

Some weights of the model checkpoint at ./NER_nor-finetuned-ner/checkpoint-500 were not used when initializing RobertaForMaskedLM: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMaskedLM were not initialized from the model checkpoint at ./NER_nor-finetuned-ner/checkpoint-500 and are newly initialized: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for pre

RobertaConfig {
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 8,
  "num_hidden_layers": 10,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.12.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 20000
}

Num parameters 52808736


In [11]:
#tokenizer_path = '/content/Norwegian_Model'
#norwegian_tokenizer = AlbertTokenizerFast.from_pretrained(tokenizer_path)

In [12]:
train_dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path=train_path,
    block_size=128,
)

C:\Users\paal_\anaconda3\lib\site-packages\transformers\data\datasets\language_modeling.py:121: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer,mlm=True, mlm_probability=0.15)

In [14]:

training_args = TrainingArguments(
    output_dir='./norBERT',
    overwrite_output_dir=True,
    num_train_epochs=3,
    learning_rate=5e-05,
    per_device_train_batch_size=batch_size,
    save_steps=len(train_dataset)/batch_size,
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)

In [15]:
trainer.train()


***** Running training *****
  Num examples = 117132
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 21963


Step,Training Loss
500,6.958200
1000,6.826400
1500,6.704600
2000,6.626400
2500,6.577900
3000,6.486700
3500,6.381900
4000,6.266000
4500,6.162600
5000,6.045400




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=21963, training_loss=5.382044451357112, metrics={'train_runtime': 15744.6819, 'train_samples_per_second': 22.318, 'train_steps_per_second': 1.395, 'total_flos': 1.1397928503671808e+16, 'train_loss': 5.382044451357112, 'epoch': 3.0})

In [ ]:
#import os
#os.environ["CUDA_LAUNCH_BLOCKING"]='1'
trainer.train()


***** Running training *****
  Num examples = 117132
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 43926


Step,Training Loss
500,7.385500
1000,6.855900
1500,6.721600
2000,6.642100
2500,6.601700
3000,6.538600
3500,6.499200
4000,6.462800
4500,6.421400
5000,6.348600


In [16]:
trainer.save_model('./norBERT')

Saving model checkpoint to ./norBERT
Configuration saved in ./norBERT\config.json
Model weights saved in ./norBERT\pytorch_model.bin


In [29]:
#test_model = RobertaForMaskedLM.from_pretrained("./norBERT/norBERT_3epochs/")
#nlp = pipeline("fill-mask", model=test_model.to('cpu'), tokenizer=tokenizer, top_k=100)
#nlp(f"jeg skjønner!... noen her på bloggen{tokenizer.mask_token}")

model = RobertaForMaskedLM.from_pretrained("./../Cloud_environment/norBERT_pretrained")
tokenizer = RobertaTokenizerFast.from_pretrained("./../Cloud_environment/norBERT_pretrained", max_len=512)

In [36]:
#nlp = pipeline("fill-mask", model=model.to('cpu'), tokenizer=tokenizer, top_k=20)
#nlp(f"Maailmassa on monenlaisia {tokenizer.mask_token}.")
#Kuinka monta ihmistä asuu Suomessa?
# Maailmassa on monenlaisia puita
# {tokenizer.mask_token}

nlp = pipeline('ner', model=model, tokenizer=tokenizer)

The model 'RobertaForMaskedLM' is not supported for ner. Supported models are ['FNetForTokenClassification', 'LayoutLMv2ForTokenClassification', 'RemBertForTokenClassification', 'CanineForTokenClassification', 'RoFormerForTokenClassification', 'BigBirdForTokenClassification', 'ConvBertForTokenClassification', 'LayoutLMForTokenClassification', 'DistilBertForTokenClassification', 'CamembertForTokenClassification', 'FlaubertForTokenClassification', 'XLMForTokenClassification', 'XLMRobertaForTokenClassification', 'LongformerForTokenClassification', 'RobertaForTokenClassification', 'SqueezeBertForTokenClassification', 'BertForTokenClassification', 'MegatronBertForTokenClassification', 'MobileBertForTokenClassification', 'XLNetForTokenClassification', 'AlbertForTokenClassification', 'ElectraForTokenClassification', 'FunnelForTokenClassification', 'MPNetForTokenClassification', 'DebertaForTokenClassification', 'DebertaV2ForTokenClassification', 'GPT2ForTokenClassification', 'IBertForTokenClas

In [6]:
!pip install datasets
!pip install conllu
!pip install seqeval
#!apt install git-lfs

In [3]:
task = "ner"
dataset = load_dataset("NbAiLab/norne", 'bokmaal')


Reusing dataset norne (C:\Users\paal_\.cache\huggingface\datasets\NbAiLab___norne\bokmaal\1.0.0\94a53c0adf84fc6ee63552a955c6435c838889e3bc240bbe4bf15938ac4b6d7d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
dataset["train"].features[f"ner_tags"]
label_list = dataset["train"].features[f"{task}_tags"].feature.names
print(label_list)

['O', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-GPE_LOC', 'I-GPE_LOC', 'B-PROD', 'I-PROD', 'B-LOC', 'I-LOC', 'B-GPE_ORG', 'I-GPE_ORG', 'B-DRV', 'I-DRV', 'B-EVT', 'I-EVT', 'B-MISC', 'I-MISC']


In [5]:

tokenizer = AutoTokenizer.from_pretrained('./../Cloud_environment/norBERT_pretrained/', add_prefix_space=True, model_max_length=512)
print(type(tokenizer))
assert isinstance(tokenizer, PreTrainedTokenizerFast)

<class 'transformers.models.roberta.tokenization_roberta_fast.RobertaTokenizerFast'>


In [6]:
tokenizer(["Hello", ",", "this", "is", "one", "sentence", "split", "into", "words", "."],  is_split_into_words=True)


{'input_ids': [0, 225, 44, 274, 427, 1999, 7997, 1524, 8595, 1335, 13632, 392, 574, 1889, 83, 7308, 87, 2045, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [7]:
example = dataset["train"][4]
print(example["tokens"])

['Deretter', 'fulgte', '"', 'Turbot', 'au', 'Four', '-', 'purée', 'de', 'chou-fleur', '"', '.']


In [8]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
print(type(tokenized_input))
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

<class 'transformers.tokenization_utils_base.BatchEncoding'>
['<s>', 'Ġ', 'D', 'er', 'etter', 'Ġfulgte', 'Ġ"', 'Ġ', 'T', 'ur', 'bot', 'Ġau', 'Ġ', 'F', 'our', 'Ġ-', 'Ġpur', 'Ã©', 'e', 'Ġde', 'Ġch', 'ou', '-', 'fle', 'ur', 'Ġ"', 'Ġ.', '</s>']


In [9]:

len(example[f"{task}_tags"]), len(tokenized_input["input_ids"])


(12, 28)

In [10]:
#print(dataset)
#dataset = dataset.rename_column("idx", "id")
#dataset = dataset.remove_columns("lang")
#dataset = dataset.remove_columns("text")
#dataset = dataset.remove_columns("lemmas")
#new_features = dataset.features.copy()
#new_features["id"] = Value('int64')
#dataset = dataset.cast(new_features)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['idx', 'lang', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 15696
    })
    validation: Dataset({
        features: ['idx', 'lang', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 2410
    })
    test: Dataset({
        features: ['idx', 'lang', 'text', 'tokens', 'lemmas', 'pos_tags', 'ner_tags'],
        num_rows: 1939
    })
})


In [11]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example[f"{task}_tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

28 28


In [12]:
#del label_all_tokens
label_all_tokens = True

In [13]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [14]:
tokenize_and_align_labels(dataset['train'][:5])


{'input_ids': [[0, 225, 48, 343, 280, 14390, 511, 312, 7236, 1161, 15615, 2], [0, 225, 47, 343, 19796, 1999, 14390, 511, 280, 5545, 505, 1113, 264, 2601, 312, 10245, 529, 360, 15195, 16373, 343, 320, 2609, 2045, 2], [0, 225, 51, 75, 300, 16841, 1393, 225, 52, 284, 2514, 388, 11715, 259, 923, 225, 49, 912, 17, 49, 284, 388, 2045, 2], [0, 225, 42, 270, 9270, 10306, 3198, 811, 225, 39, 83, 1842, 281, 942, 225, 55, 88, 18, 225, 46, 4115, 1842, 318, 225, 52, 18671, 997, 349, 225, 50, 270, 1614, 73, 811, 1999, 313, 303, 553, 9098, 3729, 19796, 312, 851, 3306, 364, 2018, 364, 518, 319, 5044, 12943, 18280, 358, 90, 1546, 316, 3627, 1999, 931, 4642, 345, 499, 309, 314, 280, 14997, 744, 1356, 19675, 12121, 621, 2045, 2], [0, 225, 40, 261, 798, 6331, 811, 225, 56, 371, 10415, 3225, 225, 42, 7785, 518, 14181, 1249, 73, 349, 962, 9939, 17, 10501, 371, 811, 2045, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],

In [15]:
tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
print(tokenized_datasets)

Loading cached processed dataset at C:\Users\paal_\.cache\huggingface\datasets\NbAiLab___norne\bokmaal\1.0.0\94a53c0adf84fc6ee63552a955c6435c838889e3bc240bbe4bf15938ac4b6d7d\cache-e2ea3642b80e1021.arrow
Loading cached processed dataset at C:\Users\paal_\.cache\huggingface\datasets\NbAiLab___norne\bokmaal\1.0.0\94a53c0adf84fc6ee63552a955c6435c838889e3bc240bbe4bf15938ac4b6d7d\cache-a594595bc76ffafe.arrow
Loading cached processed dataset at C:\Users\paal_\.cache\huggingface\datasets\NbAiLab___norne\bokmaal\1.0.0\94a53c0adf84fc6ee63552a955c6435c838889e3bc240bbe4bf15938ac4b6d7d\cache-278f2d0492cdbae8.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lang', 'lemmas', 'ner_tags', 'pos_tags', 'text', 'tokens'],
        num_rows: 15696
    })
    validation: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lang', 'lemmas', 'ner_tags', 'pos_tags', 'text', 'tokens'],
        num_rows: 2410
    })
    test: Dataset({
        features: ['attention_mask', 'idx', 'input_ids', 'labels', 'lang', 'lemmas', 'ner_tags', 'pos_tags', 'text', 'tokens'],
        num_rows: 1939
    })
})


In [19]:
#model = RobertaForTokenClassification.from_pretrained('./../Cloud_environment/norBERT_pretrained/', num_labels=len(label_list))
model = RobertaForTokenClassification.from_pretrained('./NER_nor-finetuned-ner/checkpoint-500/', num_labels=len(label_list))

In [20]:
model_name = "NOR"
batch_size = 16
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
)

In [21]:

data_collator = DataCollatorForTokenClassification(tokenizer)

In [22]:

metric = load_metric("seqeval")


In [23]:
#labels = [label_list[i] for i in example[f"{task}_tags"]]
#metric.compute(predictions=[labels], references=[labels])

In [24]:

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [25]:
#from huggingface_hub import notebook_login

#notebook_login()

In [26]:
#tokenized_dataset = {}
#tokenized_dataset['train'] = [{'id' : x['idx'], 'tokens' : x['tokens'], 'pos_tags' : x['pos_tags'], 'ner_tags' : x['ner_tags']} for x in tokenized_datasets["train"]]
print(tokenized_datasets['train'][0])


{'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'idx': '000001', 'input_ids': [0, 225, 48, 343, 280, 14390, 511, 312, 7236, 1161, 15615, 2], 'labels': [-100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100], 'lang': 'bokmaal', 'lemmas': ['lam', 'og', 'piggvar', 'på', 'bryllupsmeny'], 'ner_tags': [0, 0, 0, 0, 0], 'pos_tags': [0, 9, 0, 5, 0], 'text': 'Lam og piggvar på bryllupsmenyen', 'tokens': ['Lam', 'og', 'piggvar', 'på', 'bryllupsmenyen']}


In [27]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [28]:
trainer.train()


The following columns in the training set  don't have a corresponding argument in `RobertaForTokenClassification.forward` and have been ignored: lang, tokens, pos_tags, lemmas, ner_tags, text, idx.
***** Running training *****
  Num examples = 15696
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 2943


Epoch,Training Loss,Validation Loss


Saving model checkpoint to NOR-finetuned-ner\checkpoint-500
Configuration saved in NOR-finetuned-ner\checkpoint-500\config.json
Model weights saved in NOR-finetuned-ner\checkpoint-500\pytorch_model.bin
tokenizer config file saved in NOR-finetuned-ner\checkpoint-500\tokenizer_config.json
Special tokens file saved in NOR-finetuned-ner\checkpoint-500\special_tokens_map.json


RuntimeError: CUDA out of memory. Tried to allocate 38.00 MiB (GPU 0; 4.00 GiB total capacity; 2.39 GiB already allocated; 3.40 MiB free; 2.59 GiB reserved in total by PyTorch)

In [ ]:
trainer.evaluate()

In [ ]:
predictions, labels, _ = trainer.predict(tokenized_datasets["validation"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results